Adapted from
https://colab.research.google.com/github/huggingface/deep-rl-class/blob/master/notebooks/unit4/unit4.ipynb#scrollTo=V8oadoJSWp7C

In [1]:
#!apt install python-opengl
#!apt install ffmpeg
#!apt install xvfb
#!pip install pyvirtualdisplay
#!pip install pyglet==1.5.1
#!pip install imageio
!pip install gym[atari]
!pip install autorom[accept-rom-license]
!pip install torchvision
!pip install torch

In [2]:
# # Virtual display
# from pyvirtualdisplay import Display

# virtual_display = Display(visible=0, size=(1400, 900))
# virtual_display.start()

In [3]:
# !pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit4/requirements-unit4.txt

In [4]:
import numpy as np

from collections import deque

# PyTorch
import torch

# Gym
import gym

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
print(device)

cuda:0


In [7]:
env_id = "ALE/Pong-v5"
# Create the env
env = gym.make(env_id,obs_type="grayscale",full_action_space=False)

# Create the evaluation env
eval_env = gym.make(env_id,obs_type="grayscale",full_action_space=False)

# Get the state space and action space
s_size = gym.spaces.utils.flatten_space(env.observation_space).shape[0]
a_size = env.action_space.n

c:\Users\shwes\Projects\AI-plays-AI-generated-pong\.venv\Lib\site-packages\gym\utils\passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (210, 160)
  logger.warn(


In [8]:
print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
print("Sample observation", env.observation_space.sample()) # Get a random observation

_____OBSERVATION SPACE_____ 

The State Space is:  33600
Sample observation [[ 68 187 222 ...  13   0 231]
 [112 113 166 ...  66 144 231]
 [  7  25  86 ... 222   1 229]
 ...
 [225 175  15 ... 142  46  17]
 [245 202 109 ... 143  19  38]
 [ 30 182  79 ... 152  45 229]]


In [9]:
print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", a_size)
print("Action Space Sample", env.action_space.sample()) # Take a random action


 _____ACTION SPACE_____ 

The Action Space is:  6
Action Space Sample 0


In [10]:
from os import listdir
from os.path import isfile, join
import os
import sys

class Saver():
    """
    Class to save the output and input to an environment as tensors which can be used to train the nn
    """
    def __init__(self,save_dir,in_transform = None, out_transform = None, small_transform = None,file_limit = 1000) -> None:
        self.save_dir = save_dir

        self.observations = deque()#maxsize=3)
        self.actions = deque()#maxsize=1)
        self.rewards = deque()#maxsize=1)
        self.dones = deque()#maxsize=1)
        self.h=None
        self.w=None
        self.file_limit = file_limit
        self.in_transform = in_transform
        self.out_transform = out_transform
        self.small_transform = small_transform

        os.makedirs(os.path.dirname(save_dir), exist_ok=True)

    def add_transition(self,observation, action, reward, done):
        """
        Observations: What the agent sees (dimensions are 210 x 160)
        Actions: What the agent does (dimensions are 6 x 1)
        Rewards: The reward the agent gets (range is -1 to 1)?
        Dones: If the episode is over (True or False)
        """
        self.observations.append(observation)
        self.actions.append(action)
        self.rewards.append(reward)
        self.dones.append(done)
        if len(self.observations) == 3:
            self._save_transitions()
            #clear screen of 3 screens ago.
            self.observations.popleft()
        
        #clear old values
        if len(self.actions) == 1:
            self.actions.popleft()
        if len(self.rewards) == 1:
            self.rewards.popleft()
        if len(self.dones) == 1:
            self.dones.popleft()

    def _transform(self, input,truth):
        if self.in_transform == None or self.out_transform == None:
            return(input,truth)
        input = self.in_transform(input)
        #make the pong image bigger
        truth[0] = self.out_transform(truth[0])
        #add a small version of the pong image into index 0.
        truth = [self.small_transform(truth[0])] + truth

        return input,truth

    def _convert_to_tensors(self):
        if self.h == None or self.w == None:
            h = self.observations[0].shape[0] #height of a pong image
            w = self.observations[0].shape[1] #width of a pong image

        #take 2 images as input, return one image (the model should need two input images to determine ball velocity)
        #expand action to a matrix the size of a pong image, and add it as a seperate channel in a tensor image later
        act = np.broadcast_to(self.actions[0], (h,w) )
        rew = self.rewards[0]
        don = self.dones[0]


        input = np.stack((self.observations[0], self.observations[1], act), axis=0) #put the channels together so my life is easier and the nn pays a lot of attention to act
        truth = (self.observations[2],rew,don) #tuple of outputs
        
        

        input = torch.tensor(input,dtype=torch.float)
        truth = [
            torch.tensor(np.expand_dims(truth[0],axis=0),dtype=torch.float),
            torch.unsqueeze(torch.tensor(truth[1],dtype=torch.float),dim=0),
            torch.unsqueeze(torch.tensor(truth[2],dtype=torch.float),dim=0),
        ]
        assert input.shape == (3,h,w)
        assert truth[0].shape == (1,h,w)
        assert truth[1].shape == (1,), "shape is {}".format(truth[1].shape)
        assert truth[2].shape == (1,)

        return input,truth

    def _save_transitions(self):
        #change bool in dones to floats
        for i in range(len(self.dones)):
            self.dones[i] = float(self.dones[i])

        existing_files = [f for f in listdir(self.save_dir) if isfile(join(self.save_dir, f))]
        j = None
        if existing_files:
            #This grabs the largest integer out of all the filenames (filter the string for digit chars, convert those chars to an int)
            j = max([int(''.join([c for c in f if c.isdigit()])) for f in existing_files])
        else:
            j = -1
        j += 1

        if j >= self.file_limit:
            sys.exit(0)

        input,truth = self._convert_to_tensors()

        input,truth = self._transform(input,truth)

        torch.save(input,self.save_dir+"input{i}.pt".format(i=j))
        torch.save((truth),self.save_dir+"truth{i}.pt".format(i=j))

    

In [11]:
import torchvision
save_dir = "diffusion_training_data/"
imagenet_stats = ((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

#TODO add a transform for, and also save, a small version of the truth image.

in_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    # torchvision.transforms.Normalize(*imagenet_stats)
                                      ])
out_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
                                      ])
small_transform = out_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((64, 64)),
                                      ])
saver = Saver(save_dir,in_transform=in_transform,out_transform=out_transform, small_transform=small_transform)

In [12]:
def run_pong(n_episodes, max_t):
    for i_episode in range(1, n_episodes+1):
        state = env.reset()[0]
        for t in range(max_t):
            action = env.action_space.sample()
            state, reward, terminated, truncated, info = env.step(action)
            saver.add_transition(state, action, reward, terminated or truncated)
            if terminated or truncated:
                break 
        

In [13]:
n_episodes = 10
max_t = 1000
run_pong(n_episodes,max_t)

c:\Users\shwes\Projects\AI-plays-AI-generated-pong\.venv\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


TypeError: unsupported operand type(s) for +: 'Tensor' and 'list'